In [37]:
#Importing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.preprocessing.sequence import TimeseriesGenerator

from sklearn.preprocessing import MinMaxScaler
from statsmodels.tools.eval_measures import rmse

#ignore warnings
import warnings
warnings.filterwarnings("ignore")


import random
random.seed(100)

In [38]:
# define input sequence
series = pd.read_csv(r"C:\Users\IFMRUSER\Desktop\Deep Learning\India_covid19.csv", index_col = 'Date' , parse_dates = True)
#taking the dataset when India crossed 1000 mark 
dataset=series.iloc[67:]
dataset.head()

,Cases,Deaths,Recovered
Date,,,
2020-03-29,1024,27,95
2020-03-30,1251,32,102
2020-03-31,1397,35,123
2020-04-01,1998,58,148
2020-04-02,2543,72,191


In [39]:
#2nd order difference for Stationarty (derived from auto_arima() - see VARMA model)
dataset_transformed = dataset.diff().diff()
dataset_transformed = dataset_transformed.dropna()
dataset_transformed.head()

,Cases,Deaths,Recovered
Date,,,
2020-03-31,-81.0,-2.0,14.0
2020-04-01,455.0,20.0,4.0
2020-04-02,-56.0,-9.0,18.0
2020-04-03,-521.0,-14.0,-42.0
2020-04-04,491.0,14.0,36.0


In [40]:
# move entire data to test
print(len(dataset_transformed))
nobs=0
train = dataset_transformed.iloc[:-nobs]
test = dataset_transformed.iloc[-nobs:]
print(train.shape)
print(test.shape)

49
(0, 3)
(49, 3)


In [41]:
#Normaliznig the data
scaler = MinMaxScaler()
scaler.fit(test)
#scaled_train =  scaler.transform(train)
scaled_test =  scaler.transform(test)


In [42]:
#Defining the inputs for LSTM model
n_input = 3
n_features = 3
n_neurons = 100


train_generator = TimeseriesGenerator(scaled_test, scaled_test, length = n_input, batch_size = 4) 

In [43]:
#Building the vanilla LSTM model (this is post few hyperparameters tuning )
model = Sequential()
model.add(LSTM(n_neurons, activation = 'relu', input_shape = (n_input, n_features), recurrent_dropout = 0.45))
model.add(Dense(n_features))
model.compile(optimizer = 'adam', loss = 'mse')

In [44]:
# Fittng the model (this is post few hyperparameters tuning )
model.fit_generator(train_generator, epochs = 200, steps_per_epoch=400)

Epoch 1/200
400/400 [==============================] - 4s 11ms/step - loss: 0.0391
Epoch 2/200
400/400 [==============================] - 4s 9ms/step - loss: 0.0227
Epoch 3/200
400/400 [==============================] - 4s 9ms/step - loss: 0.0186
Epoch 4/200
400/400 [==============================] - 4s 9ms/step - loss: 0.0168
Epoch 5/200
400/400 [==============================] - 4s 9ms/step - loss: 0.0155
Epoch 6/200
400/400 [==============================] - 4s 9ms/step - loss: 0.0144
Epoch 7/200
400/400 [==============================] - 4s 9ms/step - loss: 0.0138
Epoch 8/200
400/400 [==============================] - 4s 9ms/step - loss: 0.0127
Epoch 9/200
400/400 [==============================] - 4s 9ms/step - loss: 0.0120
Epoch 10/200
400/400 [==============================] - 4s 9ms/step - loss: 0.0112
Epoch 11/200
400/400 [==============================] - 4s 9ms/step - loss: 0.0101
Epoch 12/200
400/400 [==============================] - 4s 10ms/step - loss: 0.0100
Epoch 13/20

In [45]:
scaled_test[-3:].shape

(3, 3)

In [46]:
#Predicting using the tuned model
test_predictions = []
first_eval_batch = scaled_test[-3:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(7):
    current_pred = model.predict(current_batch)[0]
    
    test_predictions.append(current_pred)
    
    current_batch = np.append(current_batch[:,1:,:], [[current_pred]], axis = 1)

In [47]:
#inversing the normalized values 
true_predictions = scaler.inverse_transform(test_predictions)
true_predictions

array([[  837.28395778,    76.89424527,   151.92095947],
       [-1897.36312267,   -47.65207443,   262.75561523],
       [ 1537.30595231,   -62.28449303,  -480.60827637],
       [ -555.90938203,   -17.64234695,   233.92785645],
       [  227.62084222,    13.1944215 ,   -14.38705444],
       [  249.01430452,   -46.73232752,  1588.8848877 ],
       [ 2070.27705657,    99.09630281,    10.87957764]])

In [51]:
#Lets see the predictions (these are the predctions in 2nd order difference)
df_cast = pd.DataFrame(true_predictions, columns=list('ABC'))
df_cast.index = pd.date_range('5/19/2020', periods=7, freq='D')
print(df_cast)

                      A          B            C
2020-05-19   837.283958  76.894245   151.920959
2020-05-20 -1897.363123 -47.652074   262.755615
2020-05-21  1537.305952 -62.284493  -480.608276
2020-05-22  -555.909382 -17.642347   233.927856
2020-05-23   227.620842  13.194421   -14.387054
2020-05-24   249.014305 -46.732328  1588.884888
2020-05-25  2070.277057  99.096303    10.879578


In [52]:
# Inversing the 2nd order difference

# Add the most recent first difference from the training set to the forecast cumulative sum
df_cast['Cases1d'] = (dataset['Cases'].iloc[-nobs-1]-dataset['Cases'].iloc[-nobs-2]) + df_cast['A'].cumsum()
df_cast['Deaths1d'] = (dataset['Deaths'].iloc[-nobs-1]-dataset['Deaths'].iloc[-nobs-2]) + df_cast['B'].cumsum()
df_cast['Recovered1d'] = (dataset['Recovered'].iloc[-nobs-1]-dataset['Recovered'].iloc[-nobs-2]) + df_cast['C'].cumsum()


# Now build the forecast values from the first difference set
df_cast['CasesForecast'] = dataset['Cases'].iloc[-nobs-1] + df_cast['Cases1d'].cumsum()
df_cast['DeathsForecast'] = dataset['Deaths'].iloc[-nobs-1] + df_cast['Deaths1d'].cumsum()
df_cast['RecoveredForecast'] = dataset['Recovered'].iloc[-nobs-1] + df_cast['Recovered1d'].cumsum()

In [53]:
#lets now see the forecasted values for next week
df_cast.iloc[:,6:9]

,CasesForecast,DeathsForecast,RecoveredForecast
2020-05-19,105795.283958,3363.894245,41822.920959
2020-05-20,109365.204793,3524.136416,44675.597534
2020-05-21,114472.431580,3622.094094,47047.665833
2020-05-22,119023.748986,3702.409425,49653.661987
2020-05-23,123802.687233,3795.919177,52245.271088
2020-05-24,128830.639785,3842.696602,56425.765076
2020-05-25,135928.869394,3988.570330,60617.138641
